# Watson-Natural Language Understanding

--------------------------------
<br>

### Descarga la presentación en [GitHub](https://github.com/Luisarg03/WatsonNLU-Twetts)
<br>

--------------------------------

### Ejemplo de implementación de aplicación Watson-Natural Language Understanding via api

#### En este ejemplo se tratara de analizar los sentimientos contenidos dentro de twitts con respecto a xxx .
#### También se analizaran las emociones contenidas dentro de twitts con respecto a xxx .
<br>

--------------------------------------------------------------------------------------------------------------

#### Que es lo que analizamos ? [Segun Wikipedia](https://es.wikipedia.org/wiki/An%C3%A1lisis_de_sentimiento)
##### Básicamente, determinar la actitud de un interlocutor o usuario con respecto a algún tema o la polaridad (negativa o positiva) contextual general de un documento.
<br>

--------------------------------------------------------------------------------------------------------------

#### Librerías especiales implementadas
<ul>
<li><strong>tweepy</strong>, para captar twitts y facilitar la autenticación via api</li>
<li><strong>ibm_watson</strong>, para analisis de sentimientos/ emociones</li>
<br>

In [1]:
import pandas as pd
import tweepy
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, CategoriesOptions

<br>

## Primera etapa: Obtener tweets
<br>

####  Autenticación en la api de twitter.

In [2]:
# Authentication
consumerKey = '-'
consumerSecret = '-'
accessToken = '-'
accessTokenSecret = '-'

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

<br>

#### Un poco de referencia del método que se utilizara para buscar twitts de los temas.
#####  [tweepy.API().search()](https://docs.tweepy.org/en/latest/api.html?highlight=tweepy.API#API.search)

##### (Metodo premium)
##### [API.search_30_day()](https://docs.tweepy.org/en/latest/api.html?highlight=API.search_30_day#API.search_30_day)
<br>

-------------------------------------------

#### Se inicia una lista vacía que se llenara con los twitts que contengan las palabras que le indiquemos al buscador.

In [3]:
list_tweets = []

### Primer tema de busqueda
search_tweets = api.search('covid', lang="es", count=100, tweet_mode='extended')

for tweet in search_tweets:
    if 'retweeted_status' in tweet._json:
        list_tweets.append(tweet._json['retweeted_status']['full_text'].upper())
        # print(tweet._json['retweeted_status']['full_text'])
    else:
        list_tweets.append(tweet.full_text.upper())
        # print(tweet.full_text)

### Quitamos los tweets spam.
list_tweets = list(set(list_tweets))

<br>

--------------------------------------------------
#### Vistazo a la lista que obtuvimos.

<br>

In [13]:
print(len(list_tweets))

86


In [5]:
for i in list_tweets:
    print(i, '\n')
    break

DESPUÉS DE MUCHO IR Y VENIR CON UNA GRAN CANTIDAD DE (DES)INFORMACIÓN SOBRE EL USO, SEGURIDAD Y UTILIDAD DE IVERMECTINA PARA TRATAR COVID-19, AYER LA PROPIA FARMACÉUTICA @MERCK, FABRICANTE DEL MEDICAMENTO, PUBLICÓ ESTE COMUNICADO ADVIRTIENDO QUE: 🧵👇🏼 HTTPS://T.CO/JLPWYAEY59 



<br>

## Segunda etapa: Análisis con Watson

<br>

###  Autenticación en la api de IBM-Watson.
#### [documentacion](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#authentication)

In [6]:
authenticator = IAMAuthenticator('-')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

### Configuración de endpoint
#### [documentacion](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#service-endpoint)

In [7]:
service = '-'
natural_language_understanding.set_service_url(service)

<br>

#### Método para analizar el sentimiento de los textos.
https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#sentiment

#### Método para clasificar los textos.
https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#categories

### Doc
[natural_language_understanding()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html)

[analyse()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=analyze#ibm_watson.natural_language_understanding_v1.NaturalLanguageUnderstandingV1.analyze)

[Features()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=features#ibm_watson.natural_language_understanding_v1.Features)

[SentimentOptions()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=sentimentoptions#ibm_watson.natural_language_understanding_v1.SentimentOptions)

[CategoriesOptions()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=ategoriesoptions#ibm_watson.natural_language_understanding_v1.CategoriesOptions)

<br>

#### Un ejemplo de como funciona este método.

In [8]:
response = natural_language_understanding.analyze(
                                                language='es',
                                                text='las naranjas son amargas y feas',
                                                features=Features(
                                                                sentiment=SentimentOptions(targets=['manzanas', 'naranjas'])
                                                                # ,categories=CategoriesOptions(limit=3)
                                                    )
                                                )


print(json.dumps(response.get_result(), indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 31,
    "features": 1
  },
  "sentiment": {
    "targets": [
      {
        "text": "naranjas",
        "score": -0.888776,
        "label": "negative"
      }
    ],
    "document": {
      "score": -0.888776,
      "label": "negative"
    }
  },
  "language": "es"
}


<br>

#### Para entender cada campo del response que obtenemos -> [Doc Response](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#analyze)

<br>

-----------------------
<br>

####  Scoring y clasificasión de tweets

<br>

In [9]:
data = []

for tweet in list_tweets:
    response = natural_language_understanding.analyze(
                                                language='es',
                                                text=tweet,
                                                features=Features(
                                                                sentiment=SentimentOptions(targets=[tweet])
                                                    )
                                                )


    json_df = response.get_result()
    json_parse = json_df["sentiment"]["targets"][0]
    data.append(json_parse)

In [10]:
df = pd.DataFrame(data)

In [11]:
df.head()

,text,score,label,mixed
0,DESPUÉS DE MUCHO IR Y VENIR CON UNA GRAN CANTI...,0.911741,positive,NaN
1,EL MUNICIPIO DE IXTAPALUCA ES MODELO DE ACCIÓN...,-0.530367,negative,NaN
2,#GRACIASPORTULUCHA EL GOBIERNO DE \n@JTOLENTIN...,-0.732547,negative,NaN
3,#GRACIASPORTULUCHA\nLAS ACCIONES IMPLEMENTADAS...,0.545318,positive,NaN
4,#CHIMALHUACÁN SIGUE EN PIE DE LUCHA ANTE EL CO...,-0.454461,negative,NaN


In [12]:
index = 13
print('TEXTO:', df['text'].iloc[index], '\n')
print('PUNTUACIÓN:', df['score'].iloc[index], '\n')
print('ETIQUETA:', df['label'].iloc[index], '\n')

TEXTO: PARIS ES UN IRRESPONSABLE, DICE Q LOS ESTUDIANTES NO SON CAPACES DE TRANSMITIR EL COVID. SIN EMBARGO, LA EVIDENCIA CIENTÍFICA SEÑALA Q NIÑOS Y JOVENES CONTRAEN Y PROPAGAN EL VIRUS SIN MOSTRAR SÍNTOMAS. POR ESTA RAZÓN, LOS ESCOLARES SON LOS QUE MÁS CONTAGIAN DENTRO DE LOS HOGARES. 

PUNTUACIÓN: -0.412403 

ETIQUETA: negative 

